# Importing library

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import cleaner
import hazm
from collections import defaultdict




## Load data

* In this block we load csv file and stop words

In [2]:
data = pd.read_csv('./culture.csv')
with open('Verb.txt') as f:
    verb = f.readlines()
verb = [unicode(i.strip(),'utf-8') for i in verb]

In [3]:
def is_verb(word):
    lemmet = hazm.Lemmatizer()
    if (('#' in lemmet.lemmatize(word)) or (word in verb)):
        return True
    return False

## Delete stop words and punctuation marks

In [30]:
def delete_stop_words(word):    
    if (word in verb):
        return False
    if (len(word)<3):
        return False
    else:
        n = re.sub(u'[\u060c|.|:|;|!|@|#|$|%|^|&|*|\xab|\xbb|/]','',word)
    return n

## Cleaning the thex

- This function tokenize the text's word and use delete_stop_words for each word to delete or keep the word in a list 
- Then return a list contain word and postion of each one

In [31]:
def clean_text(text):
    normalizer = hazm.Normalizer()
    text = normalizer.affix_spacing(text)
    sentence = nltk.word_tokenize(unicode(text,'utf-8'))
    clean_words = [] 
    for i in range(len(sentence)):
        res = delete_stop_words(sentence[i])
        if res:
            clean_words.append({'Position':i,'Word':res})
    return clean_words

## Take a sample
- Take a sample from all data and clean it . "words" variable contain the all words we need

In [32]:
text = data['mText'][6]
all_words = nltk.word_tokenize(unicode(text,'utf-8'))
words = clean_text(text)

In [33]:
lemmet = hazm.Lemmatizer()
for i in words:
    i['Word'] = lemmet.lemmatize(i['Word'])

In [34]:
position_dict= defaultdict(list)
for i in words:
    position_dict[i['Word']].append(i['Position'])
    
for i in position_dict:
    position_dict[i] = sorted(position_dict[i])

In [35]:
life_time = defaultdict(int)
for i in position_dict:
    n = position_dict[i][-1]-position_dict[i][0] -1 
    if n == -1 :
        n = 0
    life_time[i] = float(n)/len(all_words)

In [36]:
words_dict = defaultdict()
for i in position_dict:
    n = defaultdict()
#     n['Word'] = i
    n['Position'] = position_dict[i]
    n['life_time'] = life_time[i]
    n['tf'] = len(position_dict[i])
    words_dict[i] = n

In [37]:
mean = float(sum([words_dict[i]['tf'] for i in words_dict]))/len(words_dict)
for word in words_dict:

    words_dict[word]['Var'] = words_dict[word]['tf'] - mean

In [38]:
alpha = 0.55
for word in words_dict:
    words_dict[word]['Weight'] = (((1-alpha)*words_dict[word]['life_time']) + alpha*(words_dict[word]['tf']+words_dict[word]['Var'])) - words_dict[word]['Var']

    

In [39]:
sorted_list = sorted(words_dict,key = lambda x : words_dict[x]['Weight'],reverse=True)

In [40]:
w = nltk.word_tokenize(unicode(text,'utf-8'))

In [41]:
for i in sorted_list:
    print(i)
    print(words_dict[i])

هنر
defaultdict(None, {'tf': 12, 'Position': [12, 29, 52, 69, 79, 120, 132, 136, 142, 239, 247, 319], 'life_time': 0.6891891891891891, 'Weight': 2.2163395146971805, 'Var': 10.430656934306569})
جشنواره
defaultdict(None, {'tf': 10, 'Position': [11, 22, 35, 46, 178, 210, 230, 246, 259, 404], 'life_time': 0.8828828828828829, 'Weight': 2.1035016768593433, 'Var': 8.430656934306569})
داور
defaultdict(None, {'tf': 7, 'Position': [57, 91, 149, 255, 278, 352, 362], 'life_time': 0.6846846846846847, 'Weight': 1.7143124876701528, 'Var': 5.430656934306569})
نقاش
defaultdict(None, {'tf': 6, 'Position': [19, 75, 156, 174, 244, 257], 'life_time': 0.5337837837837838, 'Weight': 1.5464070822647464, 'Var': 4.430656934306569})
تجسم
defaultdict(None, {'tf': 6, 'Position': [13, 30, 47, 53, 179, 248], 'life_time': 0.527027027027027, 'Weight': 1.5433665417242066, 'Var': 4.430656934306569})
آثار
defaultdict(None, {'tf': 7, 'Position': [253, 256, 279, 318, 363, 371, 380], 'life_time': 0.28378378378378377, 'Weight

In [64]:
related_word = defaultdict(list)
for word in sorted_list[:30]:
    l_pos = 0 
    for pos in words_dict[word]['Position']:
        if pos-5 < l_pos+5:
            pos = l_pos + 10
        l_pos = pos
        related_word[word].append(w[pos-5:pos+5])

In [65]:
for i in related_word:
    t = []
    for j in related_word[i]:
        t+= j
    related_word[i] = [lemmet.lemmatize(re.sub(u'[\u060c|.|:|;|!|@|#|$|%|^|&|*|\xab|\xbb|/]','',word)).strip() for word in t ]

In [66]:
n = defaultdict()
for i in related_word:
    m = pd.Series(related_word[i]).value_counts()
    n[i] = dict(m)
    


In [67]:
probability = defaultdict(dict)
for i in n:
    for j in n[i]:
        if (j in sorted_list[:30]):
            try:
            
                prob = (float(n[i][j])/words_dict[j]['tf'])
                probability[i][j] = prob

            except:
                probability[i][j] = 0


In [68]:
for i in probability:
    sort = sorted(probability[i].items(),key=lambda x : x[1],reverse=True)
    probability[i] = sort


In [69]:
for i in probability:
    print(i)
    for j,x in probability[i]:
        print(j)
        print(x)
    print('*************')

باعث
باعث
1.0
رشته
0.2
داور
0.142857142857
هنر
0.0833333333333
*************
وهله
وهله
1.0
اساس
0.5
آثار
0.142857142857
داور
0.142857142857
*************
هنر
هنر
1.0
جدید
1.0
تجسم
0.833333333333
رشته
0.8
یازدهمین
0.666666666667
حتما
0.5
انتخاب
0.5
توجه
0.5
شاخه
0.5
نقاش
0.5
دوره
0.5
هویت
0.5
دبیر
0.5
فجر
0.5
باعث
0.333333333333
آثار
0.285714285714
وجود
0.25
جشنواره
0.2
داور
0.142857142857
*************
جشنواره
یازدهمین
1.0
جشنواره
1.0
دوره
1.0
دبیر
1.0
فجر
1.0
تجسم
0.833333333333
نقاش
0.666666666667
مراحل
0.5
دولت
0.5
جدا
0.333333333333
وجود
0.25
رشته
0.2
هنر
0.166666666667
داور
0.142857142857
آثار
0.142857142857
*************
جهانی
جهانی
1.0
*************
وجود
وجود
1.0
حتما
0.5
مراحل
0.5
داور
0.285714285714
رشته
0.2
تجسم
0.166666666667
آثار
0.142857142857
جشنواره
0.1
هنر
0.0833333333333
*************
دبیر
دبیر
1.0
یازدهمین
0.666666666667
دوره
0.5
نقاش
0.333333333333
فجر
0.25
رشته
0.2
هنر
0.166666666667
تجسم
0.166666666667
جشنواره
0.1
*************
رشته
رشته
1.0
دبیر
0.5
یازدهمین
0.333